In [1]:
import os
from PIL import Image
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path_train_data = r"E:\New folder (6)" #path where all the images are stored
path_csv = r"C:\Users\anshu\Desktop\Tensorflow-Bootcamp-master\labels.csv" #path of csv file

# This function formats the data as required for the keras flow_from_directory function.

In [3]:
def preprocessing():
    global path_train_data 
    global path_csv 
    mf = open(path_csv)
    my_file = mf.readlines()
    dog_breeds = []
    for i in range(1,len(my_file)):
        breed_name = my_file[i].split(',')[-1]
        dog_breeds.append(breed_name)
    dog_breeds = set(dog_breeds)
    dog_breeds = list(dog_breeds)  
    os.chdir(path_train_data)
    os.mkdir('train')
    os.mkdir('valid')
    os.chdir(path_train_data + r'\train')
    for i in range(0,len(dog_breeds)):
        os.mkdir(dog_breeds[i][:-1])
    os.chdir(path_train_data + r'\valid')
    for i in range(0,len(dog_breeds)):
        os.mkdir(dog_breeds[i][:-1])
    os.chdir(path_train_data)  
    im = Image.open(my_file[1].split(',')[-2] + ".jpg")
    os.chdir(path_train_data + r'\train\{}'.format(my_file[1].split(',')[-1][:-1]))
    im.save(my_file[1].split(',')[-2] + ".jpg")
    os.chdir(path_train_data)
    for i in range(2,len(my_file)):
        if i <= 8000:
            im = Image.open(my_file[i].split(',')[-2] + ".jpg")
            os.chdir(path_train_data + r'\train\{}'.format(my_file[i].split(',')[-1][:-1]))
            im.save(my_file[i].split(',')[-2] + ".jpg")
            os.chdir(path_train_data)
        else:
            im = Image.open(my_file[i].split(',')[-2] + ".jpg")
            os.chdir(path_train_data + r'\valid\{}'.format(my_file[i].split(',')[-1][:-1]))
            im.save(my_file[i].split(',')[-2] + ".jpg")
            os.chdir(path_train_data)

In [4]:
############################################################
#    preprocessing()                                        #call if your data is not formated
############################################################

# Parameters for model and training

In [5]:
img_width, img_height = 200, 200
train_data_dir = r"E:\New folder (6)\train"
validation_data_dir = r"E:\New folder (6)\valid"
nb_train_samples = 8000
nb_validation_samples = 2222
batch_size = 100
epochs = 100

# Load pretrained vgg19 model with, include_top=False because we have to add our own fully connected layer 

In [6]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0         
__________

# Freezing all the layers except "block5_conv3 and block5_conv4 (Conv2D)" because we want to fine tune the model for our problem. 

In [8]:
for layer in model.layers[:-3]:
    layer.trainable = False

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0         
__________

# Add a fully connected layer at the end of pre-trained model for our costom classes.

In [10]:
x = model.output
x = Flatten()(x)
x = Dense(1000, activation="relu")(x)              #1000 neuron in hidden layer
x = Dropout(0.5)(x)                                #50% dropout to avoid over fitting
predictions = Dense(120, activation="softmax")(x)  

# Creating final model

In [11]:
model_final = Model(input = model.input, output = predictions)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


# Define loss and optimizer for the model

In [12]:
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

# Defining data agumentations and batch generator 

In [13]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   horizontal_flip = True,
                                   fill_mode = "nearest",
                                   zoom_range = 0.3,
                                   width_shift_range = 0.3,
                                   height_shift_range=0.3,
                                   rotation_range=30
                                  )

test_datagen = ImageDataGenerator(rescale = 1./255,
                                  horizontal_flip = True,
                                  fill_mode = "nearest",
                                  zoom_range = 0.3,
                                  width_shift_range = 0.3,
                                  height_shift_range=0.3,
                                  rotation_range=30
                                 )

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (img_height, img_width),
                                                    batch_size = batch_size, 
                                                    class_mode = "categorical"
                                                   )

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        target_size = (img_height, img_width),
                                                        class_mode = "categorical"
                                                       )

Found 8000 images belonging to 120 classes.
Found 2222 images belonging to 120 classes.


# Train the model

In [ ]:
model_final.fit_generator(train_generator,
                          samples_per_epoch = nb_train_samples,
                          epochs = epochs,
                          validation_data = validation_generator,
                          nb_val_samples = nb_validation_samples
                         )

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=100, validation_data=<keras_pre..., steps_per_epoch=160, validation_steps=2222)`
  """


Epoch 1/100
